In [1]:
# Dependencies
import requests
import json
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
home_value_index_pd = pd.read_csv(
    "../Madison/Resources/Zillow Home Value Index (ZHVI).csv",
    dtype = "object",
    encoding = "utf-8"
)

# Display sample data
home_value_index_pd.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,9,0,California,state,NaN,189260.5804905432,189896.74179229472,190755.71532745418,192620.29075163053,194787.5788005574,...,736657.0613053188,729610.0999740328,722365.2612371506,717035.9787022044,716448.9319981743,719438.9021737709,724674.9614647272,731733.2131000826,739719.9182151351,747352.0519881115
1,54,1,Texas,state,NaN,110479.87192982272,110539.90954602743,110569.02873045664,110713.54982495937,110807.14395215838,...,302314.834633247,300624.99337619205,299018.04456711956,297945.50207625027,297575.3883943631,297895.17521786573,298724.90307306795,299472.14871574374,300052.0200432296,300089.9595431615
2,14,2,Florida,state,NaN,107306.76072497867,107539.19715756495,107820.95644586982,108393.70060905674,109011.34346932755,...,388123.3368840588,386773.08043296676,385629.718442275,385164.6835781837,385610.8495285095,386666.53037803964,388345.4437035078,390052.85717714886,391744.88612785895,392921.5945984132
3,43,3,New York,state,NaN,153890.06182166748,154438.06160954604,154965.59977878895,156100.19097092343,157274.08727433413,...,441114.3983949437,439438.4457567081,438046.0689251003,438141.3533454628,439650.7306595852,442121.0596610177,445024.4569522553,448103.1301720137,451232.0242803551,453887.25695120654
4,47,4,Pennsylvania,state,NaN,98186.49315737419,98396.50663041449,98593.86288165864,98996.04493311334,99409.18660154384,...,248709.46826906633,248527.00377006107,248135.2275485898,248290.1665483627,249108.5261392972,250645.91090747225,252468.69061072348,254242.20436454364,255941.7156363109,257267.7117571273


In [16]:
median_prices_pf=pd.read_csv(
    "../Madison/Resources/Median(New) - MedianPricesofExistingDetachedHomesHistoricalData.csv.csv",
    dtype = "object",
    encoding = "utf-8"
)

median_prices_pf.head(20)

,Mon-Yr,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Yuba,Unnamed: 54,Condo,LA Metro,Central Coast,Central Valley,Far North,Inland Empire,S.F. Bay Area,SoCal
0,Jan-90,"$194,952","$226,149",NaN,"$102,143",NaN,NaN,NaN,NaN,"$82,083",...,NaN,NaN,"$141,519","$203,390",NaN,NaN,NaN,NaN,"$227,366",NaN
1,Feb-90,"$196,273","$219,306",NaN,"$83,333",NaN,NaN,NaN,NaN,"$87,187",...,NaN,NaN,"$144,965","$211,024",NaN,NaN,NaN,NaN,"$234,739",NaN
2,Mar-90,"$194,856","$225,162",NaN,"$100,000",NaN,NaN,NaN,NaN,"$83,889",...,NaN,NaN,"$141,132","$209,286",NaN,NaN,NaN,NaN,"$235,337",NaN
3,Apr-90,"$196,111","$229,333",NaN,"$108,000",NaN,NaN,NaN,NaN,"$85,428",...,NaN,NaN,"$145,707","$210,302",NaN,NaN,NaN,NaN,"$233,178",NaN
4,May-90,"$195,281","$232,291",NaN,"$100,000",NaN,NaN,NaN,NaN,"$88,749",...,NaN,NaN,"$146,060","$210,148",NaN,NaN,NaN,NaN,"$235,881",NaN
5,Jun-90,"$194,410","$231,250",NaN,"$123,043",NaN,NaN,NaN,NaN,"$87,999",...,NaN,NaN,"$147,523","$209,927",NaN,NaN,NaN,NaN,"$240,386",NaN
6,Jul-90,"$193,088","$232,916",NaN,"$113,571",NaN,NaN,NaN,NaN,"$91,346",...,NaN,NaN,"$144,931","$208,582",NaN,NaN,NaN,NaN,"$241,503",NaN
7,Aug-90,"$192,180","$228,467",NaN,"$113,333",NaN,NaN,NaN,NaN,"$95,178",...,NaN,NaN,"$145,204","$199,724",NaN,NaN,NaN,NaN,"$237,727",NaN
8,Sep-90,"$189,979","$225,000",NaN,"$117,499",NaN,NaN,NaN,NaN,"$92,500",...,NaN,NaN,"$141,080","$193,597",NaN,NaN,NaN,NaN,"$225,569",NaN
9,Oct-90,"$187,630","$221,093",NaN,"$127,143",NaN,NaN,NaN,NaN,"$96,891",...,NaN,NaN,"$141,350","$199,909",NaN,NaN,NaN,NaN,"$227,917",NaN


In [15]:
median_prices_pf.columns

Index(['Mon-Yr', 'CA', 'Alameda', 'Amador', 'Butte', 'Calaveras',
       'Contra-Costa', 'Del Norte', 'El Dorado', 'Fresno', 'Glenn', 'Humboldt',
       'Kern', 'Kings', 'Lake', 'Lassen', 'Los Angeles', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Mono', 'Monterey', 'Napa', 'Nevada',
       'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento', 'San Benito',
       'San Bernardino', 'San Diego', 'San Francisco', 'San Joaquin',
       'San Luis Obispo', 'San Mateo', 'Santa Barbara', 'Santa Clara',
       'Santa Cruz', 'Shasta', 'Siskiyou', 'Solano', 'Sonoma', 'Stanislaus',
       'Sutter', 'Tehama', 'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Yolo',
       'Yuba', 'Unnamed: 54', 'Condo', 'LA Metro', 'Central Coast',
       'Central Valley', 'Far North', 'Inland Empire', 'S.F. Bay Area',
       'SoCal'],
      dtype='object')

In [22]:
#Cleaning the data 
#delete unnecessary columns 
median_prices_pf.drop(['Amador','Calaveras','Contra-Costa','Del Norte','El Dorado','Yuba','Unnamed: 54','Central Coast',
'Central Valley','Far North','Inland Empire','SoCal'],axis=1)

#Only keep data from 2018-2023
median_prices_pf["Mon-Yr"] = pd.to_datetime(median_prices_pf["Mon-Yr"],format='%b-%y')
median_prices_pf= median_prices_pf[median_prices_pf["Mon-Yr"] >= "2018-01-01"]
data = median_prices_pf.drop('Unnamed: 54',axis=1)
data

,Mon-Yr,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Yolo,Yuba,Condo,LA Metro,Central Coast,Central Valley,Far North,Inland Empire,S.F. Bay Area,SoCal
336,2018-01-01,"$527,780","$840,000","$330,000","$314,750","$306,500","$555,000","$198,500","$480,000","$245,450",...,"$383,000","$265,360","$425,000","$490,000","$605,000","$305,000","$262,000","$350,000","$800,000","$513,073"
337,2018-02-01,"$522,440","$832,000","$318,500","$297,000","$311,500","$610,000","$216,500","$468,000","$266,440",...,"$397,500","$269,000","$454,500","$498,000","$665,000","$310,000","$258,000","$350,000","$875,000","$525,000"
338,2018-03-01,"$564,830","$955,000","$340,000","$315,000","$325,000","$655,000","$200,000","$510,000","$264,950",...,"$422,500","$277,900","$466,420","$515,000","$660,000","$324,760","$252,000","$351,450","$980,000","$540,000"
339,2018-04-01,"$584,460","$969,300","$335,000","$316,000","$340,000","$689,660","$239,000","$499,999","$260,000",...,"$495,000","$285,000","$476,010","$515,000","$661,750","$330,690","$267,650","$360,000","$1,010,000","$542,000"
340,2018-05-01,"$600,860","$1,025,000","$331,500","$324,100","$329,000","$687,500","$214,000","$552,000","$279,980",...,"$476,500","$297,500","$465,000","$530,000","$695,000","$330,000","$268,000","$360,000","$1,050,000","$558,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,2023-05-01,"$836,110","$1,260,000","$449,000","$436,950","$550,000","$888,000","$315,000","$660,000","$420,000",...,"$646,730","$440,990","$635,000","$765,000","$1,000,000","$485,000","$380,000","$574,990","$1,300,000","$800,000"
401,2023-06-01,"$838,260","$1,305,000","$470,000","$413,500","$459,000","$930,000","$410,000","$700,000","$429,390",...,"$617,000","$445,000","$650,000","$775,000","$984,500","$480,000","$389,000","$570,000","$1,323,810","$815,000"
402,2023-07-01,"$832,400","$1,260,000","$442,000","$429,000","$476,500","$900,000","$349,000","$650,000","$417,500",...,"$625,000","$415,000","$645,000","$790,000","$985,000","$489,000","$376,420","$575,000","$1,255,000","$830,000"
403,2023-08-01,"$859,800","$1,250,000","$490,000","$436,250","$475,000","$844,440","$382,500","$657,000","$425,000",...,"$622,500","$449,500","$657,000","$792,500","$950,000","$485,000","$369,000","$569,990","$1,260,000","$830,000"


In [23]:
percent_change_month=pd.read_csv(
    "../Madison/Resources/Sales MTM Percent Change.csv",
    dtype = "object",
    encoding = "utf-8"
)

percent_change_month.head(50)

,Mon-Yr,CA*,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,El Dorado,Fresno,...,Yuba,Unnamed: 54,Condo,LA Metro,Central Coast,Central Valley,Far North,Inland Empire,S.F. Bay Area,SoCal
0,Jan-18,-2.7%,-38.7%,-36.5%,-28.5%,-31.6%,-28.9%,5.3%,-32.5%,-20.3%,...,-15.9%,NaN,-24.1%,-21.5%,-19.7%,-35.8%,-19.2%,-16.4%,-33.7%,-21.5%
1,Feb-18,3.3%,5.4%,15.2%,0.0%,7.5%,0.7%,-10.0%,-5.0%,8.7%,...,5.2%,NaN,5.7%,-4.0%,-12.8%,-0.1%,-5.8%,-3.6%,7.5%,-2.5%
2,Mar-18,0.3%,64.0%,34.2%,42.6%,15.1%,60.2%,5.6%,45.5%,35.5%,...,26.2%,NaN,40.2%,39.0%,51.3%,35.6%,27.2%,33.9%,56.6%,37.9%
3,Apr-18,-1.7%,9.4%,-13.7%,6.5%,-2.0%,2.6%,21.1%,8.0%,-0.7%,...,2.6%,NaN,4.9%,2.3%,2.0%,1.5%,7.1%,2.9%,6.0%,3.1%
4,May-18,-1.8%,15.5%,43.2%,1.2%,34.0%,18.9%,-17.4%,0.7%,12.0%,...,7.6%,NaN,8.5%,12.6%,14.6%,15.4%,11.3%,7.6%,17.3%,12.1%
5,Jun-18,0.4%,8.2%,19.0%,15.1%,-21.5%,0.2%,15.8%,9.4%,2.3%,...,-5.9%,NaN,-0.3%,0.9%,1.2%,-0.3%,7.3%,-1.2%,-0.8%,1.3%
6,Jul-18,-0.9%,-6.0%,-20.0%,-7.3%,2.9%,-1.6%,4.5%,8.9%,-12.0%,...,18.8%,NaN,-5.6%,-11.3%,-11.7%,-8.4%,-6.3%,-6.1%,-7.1%,-10.7%
7,Aug-18,-1.8%,-8.7%,-15.0%,14.1%,19.0%,-5.2%,-8.7%,-6.2%,7.2%,...,12.6%,NaN,2.0%,3.2%,11.0%,5.0%,14.3%,4.6%,0.3%,2.4%
8,Sep-18,-4.4%,-23.6%,13.7%,-32.2%,-19.2%,-24.9%,19.0%,-20.7%,-22.3%,...,-34.6%,NaN,-23.8%,-18.8%,-21.3%,-21.0%,-18.9%,-17.8%,-23.2%,-19.3%
9,Oct-18,3.9%,24.9%,27.6%,23.4%,14.9%,11.2%,0.0%,22.6%,8.4%,...,14.3%,NaN,14.0%,15.1%,8.8%,10.8%,12.5%,6.4%,20.1%,13.5%
